# Scheduling the doctors for the paediatric units of Hamburg
> This is an actual (albeit simpified and anonymized) case.

Employee scheduling (e.g. workers, nurses, doctors) is probably the oldest optimization problem ever. It was what George Dantzig was working on during World War II and what got him the interest of DuPont with his famous "assign 70 workers to 70 jobs" example. In this case, we are going to look at a more recent example, namely on scheduling the doctors for the paediatric units of five hospitals in Hamburg.

In particular, we have 22 doctors, each with hard constraints (regulation from pregnancy, where they can work) as well as preferences (allocation) and we have to meet the demands of all the hospitals. Let's get to it!

## Initialization and data import
First, we have to load in the data and massage it into place:
> This is a lot of parsing, but it shows what an actual program has to do before we can even start optimizing.

In [2]:
import xpress as xp
%env XPRESS=..
import pandas as pd
import numpy as np
from dataclasses import dataclass
from datetime import datetime
from datetime import timedelta
from datetime import time

# Import of the data using pandas
df = pd.read_excel("DoctorData.xlsx", sheet_name = ["Doctors", "Hospitals"])
start_time = datetime(2019,7,1)
end_time = datetime(2019,8,30)
day_limit = 12 # Number of days a doctor can work in a row
total_work_time = (end_time - start_time).days * (40/7) # How many hours a 100% allocated person would work

# Define the classes
class Doctor:
    def __init__(self, name, is_pregnant, allocation, work_locations):
        self.name = name
        self.is_pregnant = is_pregnant
        self.allocation = allocation
        self.work_locations = work_locations
                
class Shift:
    def __init__(self, start_time, end_time):
        self.start_time = start_time
        self.end_time = end_time
        self.duration_in_hours = (end_time - start_time).total_seconds() / 3600
        self.is_nightshift = start_time.time() >= time(18,0) or start_time.time() <= time(2,0)
        self.can_work_pregnant = self.duration_in_hours < 12 and not self.is_nightshift

# Create a list of hospitals
hospitals = df["Hospitals"]["ID"].tolist()

# Get the list of doctors
doctors = list()
for index, row in df["Doctors"].iterrows():
    is_pregnant = row["Pregnant"] == "Yes"
    work_list = list()
    for hospital in hospitals:
        if row[f'Work in {hospital}?'] == "Yes":
            work_list.append(hospital)
    doctors.append(Doctor(row["Name"], is_pregnant, row["Allocation [%]"]/100, work_list))
    
# Create a list of shifts
shifts = list()
shift_days = list()
current = start_time
weekday_times = [(6, 14), (14, 22), (22, 30)]
weekend_times = [(6, 18), (18, 30)]
while current <= end_time:
    shift_days.append(current)
    if current.weekday() < 5:
        shifts += [Shift(current + timedelta(hours=start), current + timedelta(hours=end))
                  for start,end in weekday_times]
    else:
        shifts += [Shift(current + timedelta(hours=start), current + timedelta(hours=end))
                  for start,end in weekend_times]
    
    current += timedelta(days=1)
        
# Finally, the index class
@dataclass(frozen=True)
class Placement:
    doctor: Doctor
    shift: Shift
    hospital: str
        
placements = [Placement(doctor, shift, hospital) for doctor in doctors for shift in shifts for hospital in hospitals]

# And another index class, related to nightshift work
@dataclass(frozen=True)
class Nightshift:
    doctor: Doctor
    day: datetime
        
nightshifts = [Nightshift(doctor, shift_day) for doctor in doctors for shift_day in shift_days]

Ok, this was a lot of parsing, but now we have all the information at our fingertips. So let's get started!

## Problem and variable definition
The degrees of freedom in our system are whether a certain doctor $d$ works in hospital $h$ during shift $s$, which we called a `Placement` above:

In [3]:
model = xp.problem("Doctor scheduling")

x = {p : xp.var(vartype = xp.binary, 
                name = f'x_{p.doctor.name}, {p.shift}, {p.hospital}') 
     for p in placements}
y = {n : xp.var(vartype = xp.integer, 
                lb = -1, ub = 1, name = f'y_{n.doctor.name},{n.day}') 
     for n in nightshifts}
t = {d : xp.var(vartype = xp.continuous, 
                lb = 0, name = f'Auxiliary for doctor {d.name}') for d in doctors}
model.addVariable(x,y,t)

## The general setup
The first part of the general setup, i.e. the shift division, is already taken care of through the definition of `shifts`. Next, we enforce though that every shift has to be worked, i.e.:
\begin{equation}
\sum \limits_{d} x_{d,s,h} = 1, \hspace{0.15cm} \forall s,h
\end{equation}

In [4]:
force_shift_fulfillment = (xp.constraint(xp.Sum(x[p] for p in placements if 
                                                p.shift == s and p.hospital == h) 
                                         == 1,
                                        name = f'Force shift fulfillment for shift {s} and hospital {h}')
                          for s in shifts for h in hospitals)

Also, due to our definition of the variable, the considereation that "All doctors are either working or have a day off" also follows naturally from the structure of the problem, and does not have to be considered further. Let's keep looking:

"A doctor can only have up to one shift a day"

This can be translated to the following equation:
\begin{equation}
\sum \limits_{s(t),h} x_{d,s,h} \leq 1, \hspace{0.15cm} \forall d,t
\end{equation}
where $t$ is the list of days in the current period, and $s(t)$ denotes a shift with the start time at the same date as $t$.

In [5]:
one_shift_limit = (xp.constraint(xp.Sum(x[p] for p in placements 
                                        if p.doctor == d and 
                                        p.shift.start_time.date() == t.date()) <= 1, 
                                name = f'One shift limit for {d.name} and day {t}')
                  for d in doctors for t in shift_days)

Lastly, we consider that the setup that a doctor can work at most 12 days in a row. In other words, if we sum up 13 consecutive days of assignments, the sum cannot exceed 12:
\begin{equation}
\sum \limits_{j=0}^{s_{\max}} \sum \limits_{h} x_{d,s(t+j),h} \leq s_{\max}, \hspace{0.15cm} \forall d,t
\end{equation}

In [6]:
max_time_working = (xp.constraint(xp.Sum(x[p] for p in placements 
                                         for j in range(day_limit + 1) 
                                         if p.doctor == d and 
                                         p.shift.start_time.date() == 
                                         (t + timedelta(days=j)).date()) <= day_limit, 
                                 name = f'Max working time for day {t} and doctor {d.name}')
                   for d in doctors for t in shift_days)

## Making it more realistic
Now that we got the general stuff out of the way, let's go a little bit more into detail and add some realism. First off, not all doctors can work in all hospitals. In other words, we have to set the variables for those doctors and hospitals to 0 for all shifts:
\begin{equation}
\sum \limits_{s,h(d)} x_{d,s,h} = 0, \hspace{0.15cm} \forall d
\end{equation}
where $h(d)$ denotes the hospitals a doctor *cannot* work at.
> Note: We could also write this out for each shift and each hospital. However, then we would add a whole lot of constraints, where this does the same job. We could though also not even generate those variables in the first place by being smarter in our `placements` definition. When putting this into production, I would probably do that.

In [7]:
hospital_limit = (xp.constraint(xp.Sum(x[p] for p in placements 
                                       if p.doctor == d and p.hospital not in d.work_locations) == 0,
                               name = f'Enforce hospital limits for doctor {d.name}')
                 for d in doctors)

Ok, the next one is that pregnant woman cannot take shifts that are in the evening or are 12 hours long. Again, in production I would make this part of the `placements` definition, but for now let's keep it here:
\begin{equation}
\sum \limits_{s(d),h} x_{d,s,h} = 0, \hspace{0.15cm} \forall d \in P
\end{equation}
where $s(d)$ are the shifts that a pregnant woman cannot work and $P$ is the set of doctors that are pregnant.

In [8]:
pregnancy_limit = (xp.constraint(xp.Sum(x[p] for p in placements if p.doctor == d 
                                        and not p.shift.can_work_pregnant) == 0,
                                name = f'Enforce pregnancy limit for doctor {d.name}') for d in doctors if d.is_pregnant)

Lastly, night shifts have to be grouped into blocks of 3 nights, with 3 nights off afterwards. This one is the trickiest constraint of the bunch, and has to be handled with a new variable $y_{d,t}$ which describes the difference between two adjacent night shifts, i.e.:
\begin{equation}
y_{d,t} = x_{d,s(t),h} - x_{d,s(t-1),h}
\end{equation}
where $s(t)$ is the shift at the night shift interval of day $t$.
> While it is possible to formulate the below without $y_{d,t}$ (i.e. by simply substituting in what we have here), it is not advised as it makes the model much harder to read.

Ok, so in fact the simple sentence in the beginning leads to two conditions:

- (a) If a doctor goes on a nightshift, he/she has to be there for 3 nights.
- (b) If a doctor goes on a nightshift, he/she cannot be assigned afterwards.

In math, this looks as follows:
\begin{align}
\sum \limits_{h,k=0}^{2} x_{d,s(t+k),h} &\geq 3y_{d,t}, \hspace{0.15cm} \forall d, t \\
\sum \limits_{h,k=3}^{5} x_{d,s(t+k),h} &\leq 3(1- y_{d,t}), \hspace{0.15cm} \forall d,t
\end{align}

In [9]:
force_nightshift_work = (xp.constraint(xp.Sum(x[p] for p in placements for k in range(3) 
                                              if p.doctor == n.doctor and p.shift.start_time.date() == (n.day.date() + timedelta(days = k))) >= 3*y[n],
                                      name = f'Force nightshift for doctor {n.doctor} at day {n.day}')
                        for n in nightshifts)
force_rest = (xp.constraint(xp.Sum(x[p] for p in placements for k in range(3,6) 
                                              if p.doctor == n.doctor and p.shift.start_time.date() == (n.day.date() + timedelta(days = k))) <= 3*(1-y[n]),
                                      name = f'Force rest for doctor {n.doctor} at day {n.day}')
                        for n in nightshifts)

model.addConstraint(force_shift_fulfillment, one_shift_limit, max_time_working, hospital_limit, 
                    pregnancy_limit, force_nightshift_work, force_rest)

## Making a good schedule
Now we get to the part of the objective function. The objective is to minimize the deviation of the actual allocation from the desired allocation, i.e.:
\begin{equation}
\left|\sum \limits_{s,h} \delta_sx_{d,s,h} - \alpha_dT\right|
\end{equation}
where $\alpha_d$ is the desired allocation of doctor $d$ and $T$ i the working time available in the period. However, we also need to resolve the absolute value as well. This leads to the following objective function:
\begin{equation}
\begin{array}{ll}
\text{minimize} & \sum \limits_{d} t_d \\
\text{subject to} & \sum \limits_{s,h} x_{d,s,h} - \alpha_dT \leq t_d \\
& \alpha_dT - \sum \limits_{s,h} x_{d,s,h} \leq t_d \\
\end{array}
\end{equation}
where $m$ is our magic number.

In [10]:
aux_definition1 = (xp.constraint(xp.Sum(p.shift.duration_in_hours*x[p] for p in placements if p.doctor == d) - 
                                 d.allocation*total_work_time <= t[d], name=f'Auxiliary 1 for {d}')
                  for d in doctors)
aux_definition2 = (xp.constraint(d.allocation*total_work_time - 
                                 xp.Sum(p.shift.duration_in_hours*x[p] for p in placements if p.doctor == d) 
                                  <= t[d], name=f'Auxiliary 2 for {d}')
                  for d in doctors)
model.addConstraint(aux_definition1, aux_definition2)

model.setObjective(xp.Sum(t[d] for d in doctors))

## Solving and post-processing

In [11]:
model.solve()
print(f'Solution status: {model.getProbStatusString()}')

Solution status: mip_optimal


To get started, let's see how the allocation target is met:

In [12]:
for d in doctors:
    work_time = sum(p.shift.duration_in_hours*model.getSolution(x[p]) for p in placements if p.doctor == d)
    print(f'{d.name}: {np.round(work_time / total_work_time,2)} | {d.allocation}')

Anne: 1.0 | 1.0
Berta: 1.04 | 0.7
Karen: 1.1 | 1.0
Matthias: 1.01 | 1.0
Thomas: 1.06 | 0.8
Ute: 1.05 | 1.0
Alexandra: 0.69 | 0.5
Mareike: 1.0 | 1.0
Paula: 1.05 | 0.7
Rafael: 0.8 | 0.8
Natalie: 0.94 | 0.5
David: 1.0 | 1.0
Rebecca: 1.01 | 1.0
Julia: 1.19 | 1.0
Maria: 1.01 | 1.0
Josephine: 0.8 | 0.8
Jana: 0.89 | 0.5
Robert: 0.7 | 0.7
Olga: 1.0 | 1.0
Sybille: 0.84 | 0.8
Martina: 1.01 | 1.0
Laura: 1.12 | 1.0


So the first conclusion we can draw from this is that the hospital is woefully understaffed: Berta would like to work 70%, but has to put in more than 100%. This is problematic, especially given that Anne is matching her 100% completely. So probably we should change our error metric to penalize outliers more, and to try to keep values below 100% actually below 100%. **How would you do this?**